In [3]:
from itertools import product

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def calc_rolling_stat(df: pd.DataFrame, month: int = 6) -> pd.DataFrame:
    """
    Calculate rolling period aggregate statistics

    Parameters
    ----------
    df: pd.DataFrame for price
        Columns: tickers
        Index: DatatimeIndex
    month: int
        Number of month as rolling window

    Returns
    -------
    pd.DataFrame
        Columns: [ret, std, ret_max, ret_min] * tickers
            ret: end-to-end return
            std: close-to-close volatility
            ret_max: maximum earn during the period
            ret_min: maximum loss during the period
        Index: DatetimeIndex
    """

    def ret(x): return np.log(x.iloc[-1] / x.iloc[0])

    def vol(x): return x.iloc[:-1].std()

    def ret_max(x): return np.log(x.max() / x.iloc[0])

    def ret_min(x): return np.log(x.min() / x.iloc[0])

    n = month * 20 + 1
    ret = df.rolling(f"{n}D").aggregate([ret, vol, ret_max, ret_min]).shift(-n)

    return ret.stack(level=0).swaplevel()

In [5]:
inf1yr = pd.read_csv("EXPINF1YR.csv", index_col = 'DATE')
inf2yr = pd.read_csv('EXPINF2YR.csv', index_col = 'DATE')
inf5yr = pd.read_csv('EXPINF5YR.csv', index_col = 'DATE')
inflation = inf1yr.join([inf2yr, inf5yr], how='outer')
inflation.index = pd.to_datetime(inflation.index)
inflation

,EXPINF1YR,EXPINF2YR,EXPINF5YR
DATE,,,
2006-01-01,2.402776,2.433645,2.417921
2006-02-01,2.508283,2.520907,2.492559
2006-03-01,1.768612,2.159393,2.374037
2006-04-01,2.683038,2.670491,2.622056
2006-05-01,2.883020,2.776154,2.676641
...,...,...,...
2023-09-01,2.812435,2.542362,2.280831
2023-10-01,2.769574,2.603669,2.412897
2023-11-01,2.791196,2.624213,2.435239


In [6]:
data = pd.read_csv("mpd_stats.csv")
ticker_def = pd.read_csv("ticker_def.csv", index_col=["ticker"])

data = data.merge(ticker_def, left_on=["market"], right_index=True, how="left")
data.loc[:, "idt"] = pd.to_datetime(data["idt"])
rate_data = data.loc[data["type"].isin(["rate", "inflation"])]

/var/folders/5d/gc4zb31n7wsdygjhh4fl1dsc0000gn/T/ipykernel_86454/2736633919.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, "idt"] = pd.to_datetime(data["idt"])


In [7]:
log_ret = pd.DataFrame(index=inflation.index)
for t, m, prob_name in [("EXPINF1YR", 6, "infl1y"), 
                        ("EXPINF2YR", 6, "infl2y"), 
                        ("EXPINF5YR", 6, "infl5y"), ]:
    close_shift = inflation.copy()
    close_shift.index = close_shift.index - pd.DateOffset(months=m)
    close_shift = close_shift.reset_index().drop_duplicates(subset="DATE", keep='last').set_index("DATE")
    idx = close_shift.index.intersection(inflation.index)
    log_ret.loc[:, prob_name] = np.log(close_shift.loc[idx, t].ffill() / inflation.loc[idx, t])
    
log_ret

/Users/kongkelly/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,infl1y,infl2y,infl5y
DATE,,,
2006-01-01,0.153181,0.128721,0.118459
2006-02-01,0.133690,0.089976,0.057217
2006-03-01,0.404856,0.192080,0.061716
2006-04-01,-0.111187,-0.076359,-0.062647
2006-05-01,-0.422583,-0.236249,-0.147030
...,...,...,...
2023-09-01,NaN,NaN,NaN
2023-10-01,NaN,NaN,NaN
2023-11-01,NaN,NaN,NaN


In [8]:
infl_data = rate_data[rate_data['market'].isin(['infl1y', 'infl2y', 'infl5y'])]

In [10]:
monthly_data = log_ret.copy()
weekly_data = infl_data.copy()
monthly_data = monthly_data.sort_index()
weekly_data = weekly_data.sort_values('idt')

new_df = pd.DataFrame(index=monthly_data.index)

# add the closest weekly_data prInc and prDec
for infl in ['infl1y', 'infl2y', 'infl5y']:
    merged = pd.merge_asof(new_df, weekly_data, left_index=True, right_on='idt', direction='backward')
    new_df[f'{infl}_p10'] = merged['p10']
    new_df[f'{infl}_p50'] = merged['p50']
    new_df[f'{infl}_p90'] = merged['p90']
    new_df[f'{infl}_log_return'] = monthly_data[infl]
new_df

,infl1y_p10,infl1y_p50,infl1y_p90,infl1y_log_return,infl2y_p10,infl2y_p50,infl2y_p90,infl2y_log_return,infl5y_p10,infl5y_p50,infl5y_p90,infl5y_log_return
DATE,,,,,,,,,,,,
2006-01-01,NaN,NaN,NaN,0.153181,NaN,NaN,NaN,0.128721,NaN,NaN,NaN,0.118459
2006-02-01,NaN,NaN,NaN,0.133690,NaN,NaN,NaN,0.089976,NaN,NaN,NaN,0.057217
2006-03-01,NaN,NaN,NaN,0.404856,NaN,NaN,NaN,0.192080,NaN,NaN,NaN,0.061716
2006-04-01,NaN,NaN,NaN,-0.111187,NaN,NaN,NaN,-0.076359,NaN,NaN,NaN,-0.062647
2006-05-01,NaN,NaN,NaN,-0.422583,NaN,NaN,NaN,-0.236249,NaN,NaN,NaN,-0.147030
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.006430,0.022615,0.046513,NaN,0.006430,0.022615,0.046513,NaN,0.006430,0.022615,0.046513,NaN
2023-10-01,0.006831,0.023215,0.047352,NaN,0.006831,0.023215,0.047352,NaN,0.006831,0.023215,0.047352,NaN
2023-11-01,0.006836,0.023251,0.047385,NaN,0.006836,0.023251,0.047385,NaN,0.006836,0.023251,0.047385,NaN


In [12]:
def calculate_probability(df, product):
    log_return_column = f'{product}_log_return'
    p10_column = f'{product}_p10'
    p50_column = f'{product}_p50'
    p90_column = f'{product}_p90'
    
    # Drop rows where any of the required values are NaN
    product_df = df[[log_return_column, p10_column, p50_column, p90_column]].dropna()
    
    p10_hit = (product_df[log_return_column] > product_df[p10_column]).sum() / product_df[log_return_column].count()
    p50_hit = (product_df[log_return_column] > product_df[p50_column]).sum() / product_df[log_return_column].count()
    p90_hit = (product_df[log_return_column] > product_df[p90_column]).sum() / product_df[log_return_column].count()
    
    return pd.Series([p10_hit, p50_hit, p90_hit], index=['p10', 'p50', 'p90'])


products = ['infl1y', 'infl2y', 'infl5y']
hittest1 = pd.DataFrame({product: calculate_probability(new_df, product) for product in products})
hittest1

,infl1y,infl2y,infl5y
p10,0.49375,0.530488,0.542683
p50,0.48750,0.487805,0.481707
p90,0.47500,0.451220,0.457317


In [17]:
import quandl
quandl.ApiConfig.api_key = '_kjoBVYapaKUSUA9yWjn'
treasury = quandl.get('USTREASURY/REALYIELD')
treasury.index = pd.to_datetime(treasury.index)
treasury

,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,
2003-01-02,1.75,2.24,2.43,NaN,NaN
2003-01-03,1.75,2.25,2.43,NaN,NaN
2003-01-06,1.79,2.27,2.46,NaN,NaN
2003-01-07,1.76,2.24,2.42,NaN,NaN
2003-01-08,1.68,2.14,2.29,NaN,NaN
...,...,...,...,...,...
2023-12-29,1.72,1.72,1.72,1.82,1.90
2024-01-02,1.76,1.75,1.74,1.84,1.91
2024-01-03,1.73,1.71,1.71,1.81,1.89


In [28]:
coupon_rate = 0.02
def bond_price(yield_rate, maturity_years, face_value=100, coupon_rate=coupon_rate):
    period_rate = yield_rate / 2
    total_periods = maturity_years * 2
    coupon_payment = face_value * (coupon_rate / 2)
    price = sum(coupon_payment / (1 + period_rate)**(i+1) for i in range(total_periods)) + face_value / (1 + period_rate)**total_periods
    return price

treasury['5YR_Price'] = treasury['5 YR'].apply(lambda y: bond_price(y/100, 5))
treasury['10YR_Price'] = treasury['10 YR'].apply(lambda y: bond_price(y/100, 10))
treasury.index = pd.to_datetime(treasury.index)
treasury

,5 YR,7 YR,10 YR,20 YR,30 YR,5YR_Price,10YR_Price
Date,,,,,,,
2003-01-02,1.75,2.24,2.43,NaN,NaN,101.191891,96.202920
2003-01-03,1.75,2.25,2.43,NaN,NaN,101.191891,96.202920
2003-01-06,1.79,2.27,2.46,NaN,NaN,101.000112,95.944080
2003-01-07,1.76,2.24,2.42,NaN,NaN,101.143907,96.289373
2003-01-08,1.68,2.14,2.29,NaN,NaN,101.528497,97.421204
...,...,...,...,...,...,...,...
2023-12-29,1.72,1.72,1.72,1.82,1.90,101.335996,102.562349
2024-01-02,1.76,1.75,1.74,1.84,1.91,101.143907,102.376916
2024-01-03,1.73,1.71,1.71,1.81,1.89,101.287935,102.655206


In [29]:
log_ret1 = pd.DataFrame(index=treasury.index)
for t, m, prob_name in [("5YR_Price", 6, "tr5yr"), 
                        ("10YR_Price", 6, "tr10yr")]:
    close_shift = treasury.copy()
    close_shift.index = close_shift.index - pd.DateOffset(months=m)
    close_shift = close_shift.reset_index().drop_duplicates(subset="Date", keep='last').set_index("Date")
    idx = close_shift.index.intersection(treasury.index)
    log_ret1.loc[:, prob_name] = np.log(close_shift.loc[idx, t].ffill() / treasury.loc[idx, t])
    
log_ret1

,tr5yr,tr10yr
Date,,
2003-01-02,0.033751,0.042320
2003-01-03,0.030417,0.030594
2003-01-06,NaN,NaN
2003-01-07,0.027082,0.025190
2003-01-08,0.020908,0.015306
...,...,...
2023-12-29,NaN,NaN
2024-01-02,NaN,NaN
2024-01-03,NaN,NaN


In [33]:
tr_data = rate_data[rate_data['market'].isin(['tr5yr', 'tr10yr'])]

,market,idt,maturity_target,mu,sd,skew,kurt,p10,p50,p90,lg_change_decr,prDec,lg_change_incr,prInc,name,type
10011,tr10yr,2013-10-17,NaN,0.00088,0.02821,-0.59321,0.79563,-0.03580,0.00305,0.03401,-5.0,0.05314,5.0,0.02285,10-Year Treasury price (CME),rate
10012,tr10yr,2013-10-31,NaN,0.00082,0.02595,-0.63180,1.01019,-0.03281,0.00323,0.03090,-5.0,0.04161,5.0,0.01495,10-Year Treasury price (CME),rate
10013,tr10yr,2013-11-14,NaN,0.00119,0.02547,-0.74692,1.03489,-0.03242,0.00431,0.03013,-5.0,0.04143,5.0,0.00986,10-Year Treasury price (CME),rate
10014,tr10yr,2013-11-27,NaN,0.00118,0.02737,-0.65517,0.76711,-0.03531,0.00417,0.03238,-5.0,0.05029,5.0,0.01704,10-Year Treasury price (CME),rate
10015,tr10yr,2013-12-12,NaN,0.00110,0.02775,-0.54148,0.47768,-0.03632,0.00359,0.03374,-5.0,0.04977,5.0,0.02075,10-Year Treasury price (CME),rate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11014,tr5yr,2023-12-13,3.0,-0.00045,0.02757,0.33532,0.69462,-0.03345,-0.00202,0.03449,-3.0,0.12885,3.0,0.12523,5-Year Treasury price (CME),rate
11015,tr5yr,2023-12-20,3.0,-0.00073,0.02818,0.56406,0.82003,-0.03361,-0.00373,0.03583,-3.0,0.13202,3.0,0.13093,5-Year Treasury price (CME),rate
11016,tr5yr,2023-12-27,3.0,-0.00078,0.02893,0.73601,1.07895,-0.03331,-0.00510,0.03801,-3.0,0.13128,3.0,0.13820,5-Year Treasury price (CME),rate
11017,tr5yr,2024-01-03,3.0,-0.00059,0.02982,0.44560,0.61526,-0.03649,-0.00299,0.03789,-3.0,0.15264,3.0,0.14184,5-Year Treasury price (CME),rate


In [36]:
daily_data = log_ret1.copy()
weekly_data = tr_data.copy()
daily_data = daily_data.sort_index()
weekly_data = weekly_data.sort_values('idt')

new_df1 = pd.DataFrame(index=weekly_data.index)

for tr in ['tr5yr', 'tr10yr']:
    merged = pd.merge_asof(weekly_data, daily_data, left_on='idt', right_index=True, direction='backward')
    new_df1[f'{tr}_p10'] = weekly_data['p10']
    new_df1[f'{tr}_p50'] = weekly_data['p50']
    new_df1[f'{tr}_p90'] = weekly_data['p90']
    new_df1[f'{tr}_log_return'] = merged[tr]
new_df1

,tr5yr_p10,tr5yr_p50,tr5yr_p90,tr5yr_log_return,tr10yr_p10,tr10yr_p50,tr10yr_p90,tr10yr_log_return
10011,-0.03580,0.00305,0.03401,-0.010570,-0.03580,0.00305,0.03401,-0.006415
10012,-0.03281,0.00323,0.03090,NaN,-0.03281,0.00323,0.03090,NaN
10013,-0.03242,0.00431,0.03013,-0.005288,-0.03242,0.00431,0.03013,0.012835
10014,-0.03531,0.00417,0.03238,0.001923,-0.03531,0.00417,0.03238,0.024752
10015,-0.03632,0.00359,0.03374,0.009122,-0.03632,0.00359,0.03374,0.032962
...,...,...,...,...,...,...,...,...
11016,-0.03331,-0.00510,0.03801,NaN,-0.03331,-0.00510,0.03801,NaN
10530,-0.05387,0.00361,0.04893,NaN,-0.05387,0.00361,0.04893,NaN
11017,-0.03649,-0.00299,0.03789,NaN,-0.03649,-0.00299,0.03789,NaN
10531,-0.05674,0.00583,0.04902,NaN,-0.05674,0.00583,0.04902,NaN


In [38]:
products = ['tr5yr', 'tr10yr']
hittest2 = pd.DataFrame({product: calculate_probability(new_df1, product) for product in products})
hittest2

,tr5yr,tr10yr
p10,0.823529,0.703663
p50,0.452830,0.457270
p90,0.178690,0.249723
